In [1]:
import numpy as np  
import re  
import nltk  
from sklearn.datasets import load_files  
nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords  

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\achintj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\achintj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import pandas as pd

In [4]:
movie_data = load_files(r"D:\MyWork\DSWork\Project - Resume Ranker\txt_sentoken")

In [5]:
X,Y = movie_data.data,movie_data.target

In [6]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document) 

In [50]:
documents[0]

'arnold schwarzenegger ha been an icon for action enthusiast since the late 80 but lately his film have been very sloppy and the one liner are getting worse nit hard seeing arnold a mr freeze in batman and robin especially when he say ton of ice joke but hey he got 15 million what it matter to him nonce again arnold ha signed to do another expensive blockbuster that can compare with the like of the terminator series true lie and even eraser nin this so called dark thriller the devil gabriel byrne ha come upon earth to impregnate woman robin tunney which happens every 1000 year and basically destroy the world but apparently god ha chosen one man and that one man is jericho cane arnold himself nwith the help of trusty sidekick kevin pollack they will stop at nothing to let the devil take over the world nparts of this are actually so absurd that they would fit right in with dogma nyes the film is that weak but it better than the other blockbuster right now sleepy hollow but it make the wo

In [7]:
from sklearn.feature_extraction.text import CountVectorizer  
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = vectorizer.fit_transform(documents).toarray()  

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer  
tfidfconverter = TfidfTransformer()  
X = tfidfconverter.fit_transform(X).toarray() 

In [9]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0) 

In [10]:
classifier = RandomForestClassifier(n_estimators=5000, random_state=1,n_jobs=-1)  
classifier.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5000, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [11]:
y_pred = classifier.predict(X_test)  

In [12]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 

[[179  29]
 [ 31 161]]
              precision    recall  f1-score   support

           0       0.85      0.86      0.86       208
           1       0.85      0.84      0.84       192

   micro avg       0.85      0.85      0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

0.85


In [15]:
classifier = MultinomialNB()  
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test)  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 

[[160  48]
 [ 34 158]]
              precision    recall  f1-score   support

           0       0.82      0.77      0.80       208
           1       0.77      0.82      0.79       192

   micro avg       0.80      0.80      0.80       400
   macro avg       0.80      0.80      0.79       400
weighted avg       0.80      0.80      0.80       400

0.795


In [36]:
classifier = SVC(kernel='rbf',C=1000, gamma='auto')  
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test)  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 

[[168  40]
 [ 28 164]]
              precision    recall  f1-score   support

           0       0.86      0.81      0.83       208
           1       0.80      0.85      0.83       192

   micro avg       0.83      0.83      0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400

0.83


In [28]:
classifier = LogisticRegression(solver = 'lbfgs')  
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test)  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 

[[168  40]
 [ 28 164]]
              precision    recall  f1-score   support

           0       0.86      0.81      0.83       208
           1       0.80      0.85      0.83       192

   micro avg       0.83      0.83      0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400

0.83


In [37]:
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
res = cosine_similarity(X)

In [39]:
res.shape

(2000, 2000)

In [45]:
d = pd.Series(data =res[929:930][0])

In [46]:
d.sort_values(ascending=False)[:10]

929     1.000000
620     0.976600
1028    0.371128
650     0.364354
857     0.271096
696     0.265979
901     0.257583
1298    0.257534
40      0.256231
1833    0.246169
dtype: float64

In [48]:
print(movie_data.data[929])

b'it rocks-actually , lots of rocks fly at us or from us , in slow or fast motion , at several points in the film . \nthey seem like dangerous rocks because they kind of twirl through the air instead of just propelling forward , and when they land-once in a while , when we need a break from the space sequences-they cause damage enough to destroy the chrysler building and the like . \n ( nary a mention of these apocalyptic events is made after they occur . ) \nthey also just might be the most interesting element of armageddon , a steroid user\'s answer to deep impact . \nbruce willis stars as harry stamper , a famed oil-driller commissioned by the white house and nasa to stop a giant asteroid before it travels beyond " zero barrier " and destroys our planet . \nwhy an oil driller ? \nthey require someone experience with deep-core mining to plant a nuclear missile into said asteroid . \n ( in one unintentionally ( ? ) \nhilarious sequence , nasa asks harry to inspect a deep-core driller 

In [49]:
movie_data.data[620]

b'new address . \nsame old attitude . \ndon\'t forget to recommend a film , read the journal , or send me some of that nasty hate mail . ) \nstarring bruce willis , billy bob thornton , liv tyler , ben affleck written by jonathan hensleigh and j . j . \nabrams directed by michael bay \nit rocks-actually , lots of rocks fly at us or from us , in slow or fast motion , at several points in the film . \nthey seem like dangerous rocks because they kind of twirl through the air instead of just propelling forward , and when they land-once in a while , when we need a break from the space sequences-they cause damage enough to destroy the chrysler building and the like . \n ( nary a mention of these apocalyptic events is made after they occur . ) \nthey also just might be the most interesting element of armageddon , a steroid user\'s answer to deep impact . \nbruce willis stars as harry stamper , a famed oil-driller commissioned by the white house and nasa to stop a giant asteroid before it trav